This is the very fast vectorized solver for the explicit, leapfrog scheme. This vectorization will also speed up the code on a parallel, multi-core system. Not only are there no loops over space in the code, but there are no matrices that need to be assembled and stored, not even sparse ones.

In [1]:
import numpy as np
from numpy import newaxis

def solver(U0, U1, f, c, Lx, Ly, Nx, Ny, dt, T, exact_solution):

    x = np.linspace(0, Lx, Nx+1)  # Mesh points in x dir
    y = np.linspace(0, Ly, Ny+1)  # Mesh points in y dir
    # Make sure dx, dy, and dt are compatible with x, y and t
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    #dt = t[1] - t[0]

    xv = x[:,newaxis]          # For vectorized function evaluations
    yv = y[newaxis,:]

    stability_limit = (1/float(c))*(1/np.sqrt(1/dx**2 + 1/dy**2))
    if dt <= 0:                # max time step?
        safety_factor = -dt    # use negative dt as safety factor
        dt = safety_factor*stability_limit
    elif dt > stability_limit:
        print ('error: dt=%g exceeds the stability limit %g' % \
              (dt, stability_limit))
    Nt = int(round(T/float(dt)))
    t  = np.linspace(0, Nt*dt, Nt+1)    # mesh points in time
    dt = t[1] - t[0]  # to avoid rounding errors
    Cx2 = (c*dt/dx)**2;  Cy2 = (c*dt/dy)**2    # helper variables
    dt2 = dt**2

    u     = np.zeros((Nx+1,Ny+1))  # Solution array
    u_n   = np.zeros((Nx+1,Ny+1))  # Solution at t-dt
    u_nm1 = np.zeros((Nx+1,Ny+1))  # Solution at t-2*dt
    f_a   = np.zeros((Nx+1,Ny+1))  # For compiled loops

    Ix = range(0, u.shape[0])
    Iy = range(0, u.shape[1])
    It = range(0, t.shape[0])

    import time; t0 = time.process_time()  # For measuring CPU time

    # Load initial condition into u_n
    u_n[:,:] = U0(xv, yv)

    n = 0
    # First step requires a special formula
    f_a[:,:] = f(xv, yv, t[n])  # precompute, size as u
    U1_a     = U1(xv, yv)
    u_xx = u_n[:-2,1:-1] - 2*u_n[1:-1,1:-1] + u_n[2:,1:-1]
    u_yy = u_n[1:-1,:-2] - 2*u_n[1:-1,1:-1] + u_n[1:-1,2:]
    u[1:-1,1:-1] = u_n[1:-1,1:-1] +  dt*U1_a[1:-1, 1:-1] + \
                   0.5*Cx2*u_xx + 0.5*Cy2*u_yy + 0.5*dt2*f_a[1:-1,1:-1]
    # Boundary condition u=0
    j = 0 ;             u[:,j] = 0
    j = u.shape[1]-1 ;  u[:,j] = 0
    i = 0 ;             u[i,:] = 0
    i = u.shape[0]-1 ;  u[i,:] = 0
    # Update solution for next step
    #u_nm1[:] = u_n;  u_n[:] = u  # safe, but slow
    u_nm1, u_n, u = u_n, u, u_nm1
    # Loop over time
    for n in It[1:-1]:
        f_a[:,:] = f(xv, yv, t[n])  # precompute, size as u
        u_xx = u_n[:-2,1:-1] - 2*u_n[1:-1,1:-1] + u_n[2:,1:-1]
        u_yy = u_n[1:-1,:-2] - 2*u_n[1:-1,1:-1] + u_n[1:-1,2:]
        u[1:-1,1:-1] = 2*u_n[1:-1,1:-1] - u_nm1[1:-1,1:-1] + \
                       Cx2*u_xx + Cy2*u_yy + dt2*f_a[1:-1,1:-1]
        # Boundary condition u=0
        j = 0 ;             u[:,j] = 0
        j = u.shape[1]-1 ;  u[:,j] = 0
        i = 0 ;             u[i,:] = 0
        i = u.shape[0]-1 ;  u[i,:] = 0
        # Update solution for next step
        u_nm1, u_n, u = u_n, u, u_nm1

    # Error at t=T
    u_e  = exact_solution(xv, yv, t[n])
    diff = abs(u_nm1 - u_e).max()
    print ('Error at time t = %g is %g' % (T, diff))
    # Important to set u = u_n if u is to be returned!
    t1 = time.process_time()
    # dt might be computed in this function so return the value
    return dt, t1 - t0

This is the definition of the quadratic test problem, for which the numerical solution should be exact, to within machine precision. 

In [2]:
def quadratic(Nx, Ny):

    def exact_solution(x, y, t):
        return x*(Lx - x)*y*(Ly - y)*(1 + 0.5*t)

    def U0(x, y):
        return exact_solution(x, y, 0)

    def U1(x, y):
        return 0.5*exact_solution(x, y, 0)

    def f(x, y, t):
        return 2*c**2*(1 + 0.5*t)*(y*(Ly - y) + x*(Lx - x))

    Lx = 5;  Ly = 2
    c = 1.5
    dt = -1 # use longest possible steps
    T = 18

    new_dt, cpu = solver(
        U0, U1, f, c, Lx, Ly, Nx, Ny, dt, T, exact_solution)
    return new_dt, cpu

In [4]:
quadratic(20,20)

Error at time t = 18 is 1.20792e-13


(0.06189844605901729, 0.038959999999999884)

In [5]:
quadratic(40,40)

Error at time t = 18 is 2.13163e-13


(0.030949223029508643, 0.09991799999999995)

In [6]:
18/.0309

582.5242718446602

In [11]:
quadratic(100,100)

Error at time t = 18 is 3.48166e-13


(0.012379689211803458, 0.51606)

In [8]:
18/0.01238

1453.9579967689822